In [1]:
# downloading dataset from kaggle
import modin.pandas as pd
import os
import opendatasets as od

In [5]:
# assign kaggle dataset link
# open kaggle account. Navigate to settings and search for api tok generation. Generate a new api token or use the previously created token if any.
dataset_url = "https://www.kaggle.com/datasets/bmadushanirodrigo/fracture-multi-region-x-ray-data"

In [8]:
prefix_url = "https://www.kaggle.com/datasets/"
dataset_username, dataset_id = dataset_url.split("/")[4:]

bmadushanirodrigo fracture-multi-region-x-ray-data


In [ ]:
# using opendataset to download dataset from kaggle
od.download(prefix_url+dataset_username+'/'+dataset_id)

# Data Ingestion Code Base

In [1]:
import os
%pwd

'c:\\Users\\naura\\Desktop\\end-to-end-ML\\computer-vision\\Bone-fracture-xray-classification\\notebooks'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\naura\\Desktop\\end-to-end-ML\\computer-vision\\Bone-fracture-xray-classification'

In [3]:
# Entity defined

from pydantic import BaseModel
from pathlib import Path

# model attribute values are defined in config.yaml
class DataIngestionConfig(BaseModel):
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from boneFractureClassification.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from boneFractureClassification.utils.common import read_yaml, create_directories

In [5]:
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # calling config/config.yaml attribute "artifacts_root" which returns artifacts as value which is a folder name to be created.
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        # calling root_dir attribute from config/config.yaml which returns path
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [6]:
import os
import zipfile
import opendatasets as od
from boneFractureClassification import logger
from boneFractureClassification.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self) -> str:
        """_summary_

        Returns:
            str: _description_
        """
        
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading dataset from {dataset_url} into {zip_download_dir}")
            
            user_name, dataset_id = dataset_url.split("/")[4:]
            prefix_url = "https://www.kaggle.com/datasets/"
            od.download(prefix_url+user_name+"/"+dataset_id,data_dir=zip_download_dir)
            
            logger.info(f"Finished Downloading!. Data stored at {zip_download_dir}")
        except Exception as e:
            raise e
    
    """    
    # extract zip file 
    def extract_zip(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as f:
            f.extractall(unzip_path)
    """

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2024-05-30 23:55:09,715: INFO: common: yaml file: config\config.yaml loaded successfully!]
[2024-05-30 23:55:09,719: INFO: common: yaml file: params.yaml loaded successfully!]
[2024-05-30 23:55:09,727: INFO: common: created directory at: artifacts]
[2024-05-30 23:55:09,731: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-30 23:55:09,739: INFO: 242197021: Downloading dataset from https://www.kaggle.com/datasets/bmadushanirodrigo/fracture-multi-region-x-ray-data into artifacts\data_ingestion\data.zip]
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/bmadushanirodrigo/fracture-multi-region-x-ray-data


100%|██████████| 481M/481M [01:30<00:00, 5.57MB/s] 



[2024-05-30 23:57:26,933: INFO: 242197021: Finished Downloading!. Data stored at artifacts\data_ingestion\data.zip]
